<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Programmation_sous_MapReduceTP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction à la programmation distribuée sous <i>MapReduce</i>
L'objet principal de ce notebook est de maîtriser la programmation de traitement distribué sous <b>MapReduce</b>. 
Pour rappel, sous Hadoop, il ne revient pas au programmeur applicatif de mettre en œuvre les mécanismes de réplication, ni de gérer la traçabilité des grains et leur réaffectation. Ces tâches sont de la responsabilité du framework. 

Le programmeur applicatif a néanmoins le rôle de reformuler les algorithmes qu'il souhaite mettre en œuvre sur la plateforme distribuée suivant le mécanisme d’exécution <i>MapReduce</i>, un mécanisme d’exécution très populaire, présent dans plusieurs frameworks distribués.

<i>MapReduce</i> décompose l’ensemble des opérations à réaliser en deux types de tâches <b>élémentaires</b> et <b>uniformes</b>, les <i>Map</i> et les <i>Reduce</i>. Chaque donnée passe d'abord par une tâche <i>Map</i> qui la transforme et éventuellement par une seconde tâche <i>Reduce</i>. 

Aucun ordre d'exécution particulier n'est attendu entre différentes tâches <i>Map</i> ou entre différentes tâches <i>Reduce</i>. Une ou plusieurs tâches <i>Map</i> et/ou <i>Reduce</i> peuvent être facilement assignées à chaque nœud de calcul. Un nœud de calcul peut correspondre à un ordinateur individuel ou à un cœur d'une unité centrale multi-cœur. Dans ce dernier cas, la mémoire vive et le stockage de masse de l’ordinateur sont partagés entre les cœurs.

Le fonctionnement général de MapReduce est constitué des étapes suivantes:
<ol>
<li>L'ensemble de données à traiter est découpé en fragments (<i>chunks</i>).</li>
<li>Chaque tâche <i>Map</i> est assignée à un nœud de calcul qui reçoit un ou plusieurs fragments que la tâche <i>Map</i> transforme en une séquence de paires \[clé, valeur].</li>
<li>Chaque tâche <i>Reduce</i> est associée à une ou plusieurs clés et est assignée à un nœud de calcul.</li>
<li>Les paires (clé, valeur) produites par les <i>Map</i> sont groupées par clés et stockées sur les nœuds de calcul qui exécuteront les tâches <i>Reduce</i> respectives (étape shuffle).</li>
<li>Chaque tâche <i>Reduce</i> combine, pour chaque clé qui lui est associée, les valeurs des paires [clé, valeur] avec cette clé ; les résultats sont stockés et constituent le résultat du traitement.</li>
</ol>

Le programmeur écrit les fonctions <i>Map</i> et <i>Reduce</i>, le framework se charge du reste comme illustré ci-dessous dans le décompte distribué de la fréquence de chaque mot d'un corpus.
<!-- img width="70%" src="https://res.cloudinary.com/talend/image/upload/q_auto,w_923,h_486/resources/seo-articles/seo-what-is-mapreduce_gj9ehi.webp" -->

<img width="70%" src="https://www.nayaa.fr/bigdata/mr-execution-ex.png">

##Installation du Java Development Kit (JDK) 
Hadoop est écrit en Java et nécessite donc l'installation d'exécution de Java.

Installation du JDK

In [39]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [40]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [41]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

--2023-01-14 10:48:13--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz.1’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  82.3MB/s    in 6.4s    

2023-01-14 10:48:19 (75.0 MB/s) - ‘hadoop-3.3.0.tar.gz.1’ saved [500749234/500749234]



Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [43]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [44]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [45]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  201M  100  201M    0     0   101M      0  0:00:01  0:00:01 --:--:--  176M


Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [46]:
!mv purchases.txt ~/myinput/purchases.txt

Vérification que les fichiers ont été bien copiés

In [47]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [53]:
!head -20  ~/myinput/purchases.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard
2012-01-01	09:00	Stockton	Men's Clothing	247.18	MasterCard
2012-01-01	09:00	Austin	Cameras	379.6	Visa
2012-01-01	09:00	New York	Consumer Electronics	296.8	Cash
2012-01-01	09:00	Corpus Christi	Toys	25.38	Discover
2012-01-01	09:00	Fort Worth	Toys	213.88	Visa
2012-01-01	09:00	Las Vegas	Video Games	53.26	Visa
2012-01-01	09:00	Newark	Video Games	39.75	Cash
2012-01-01	09:00	Austin	Cameras	469.63	MasterCard
2012-01-01	09:00	Greensboro	DVDs	290.82	MasterCard
2012-01-01	09:00	San Francisco	Music	260.65	Discover
2012-01-01	09:00	Lincoln	Garden	136.9	Visa
2012-01-01	09:00	Buffalo	Women's Clothing	483.82	Visa
2012-01-01	09:00	San Jose	Women's Clothing	215.82	Cash
2012-01-01	09:00	Boston	Cameras	418.94	Amex
2012-01-01	09:00	Houston

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Activité 1

Le travail consiste à utiliser <i>MapReduce</i> avec le langage Python et effectuer un traitement distribué. Notre but est de déterminer le total des achats par magasin en exploitant les données du fichier <purchases.txt>.

Vous devrez implémenter les fonctions <i>Map<i> et <i>Reduce</> du traitement.

Contenu de traitement dans la phase \<map>. <h2>C'est à vous de le faire &#8987;</h2>

In [55]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(store, "\t", str(amount))

In [65]:
#@title
#!/usr/bin/python
import sys
# Mettez vos instructions ...
def mapper(file_path):
  fichier = open(file_path, 'r')
  file_line = fichier.readlines()
  fichier.close()
  result = []

  for line in file_line:
    data = line.strip().split("\t")
    if len(data) == 6:
      date, time, store, item, montant, payment = data
      result.append((store, montant))
      #print(store, "\t", str(montant))
  
  return result

Sauvegarde du code de traitement de la phase <map> dans le fichier "/content/map.py"

In [57]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[55]) 
f.close()

Attribution de permission d'accès et d'exécution sur le fichier <mapper.py>

In [58]:
!chmod u+rwx /content/mapper.py

Test du traitement de la phase <map> sur quelques enregistrements

In [59]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

San Jose 	 214.05
Fort Worth 	 153.57
San Diego 	 66.08
Pittsburgh 	 493.51
Omaha 	 235.63
Stockton 	 247.18
Austin 	 379.6
New York 	 296.8
Corpus Christi 	 25.38
Fort Worth 	 213.88


Contenu de traitement de la phase \<reduce>. <h2>C'est à vous de le faire aussi &#128521;</h2>

In [60]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += float (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [ ]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...

def reducer(file_path):
  map = mapper(file_path)
  map_new = {}

  for elm in map:
    if elm[0] not in map_new:
      map_new.update({elm[0] : 0})

  for key in map:
    map_new.update({key[0] : float(map_new[key[0]]) + float(key[1])})

  print("Magazin\t\tMontant Total")
  print("##################################")
  print(len(map_new))
  for elm in map_new:
    print(elm, "\t", map_new[elm])


file_path = '/root/myinput/purchases.txt'
reducer(file_path)

Sauvegarde du code de traitement de la phase <reduce> dans le fichier "/content/reduce.py"

In [61]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[60]) 
f.close()

Attribution de permission d'accès et d'exécution sur le fichier <reducer.py>

In [70]:
!chmod u+rwx /content/reducer.py

Test du traitement distribué sur quelques enregistrements

In [ ]:
!head -70000 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Lancement d'un job entier. Le résultat est dans le dossier "~/tryout".

In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

Affichage du contenu du dossier "~/myoutput".

In [72]:
!ls ~/myoutput

part-00000  _SUCCESS


Affichage d'un part du résultat contenu dans le fichier de sortie. On y trouve la fréquence de chaque mot contenu dans le corpus de documents.

In [ ]:
!tail -n 30 ~/myoutput/part-00000

# Activité 2
Nous continuons à travailler avec le même fichier en entrées (purchases.txt), mais pour obtenir des résultats différents. <u>Le but est donc d’écrire vos propres Mappers et Reducers</u>.
<ol>
<li>Donner le nombre de paiement par mode de paiement.</li>
<li>Quel est le chiffre d'affaire réalisé selon les jours de la semaine ?</li>
<li>Quelle est la liste des magasins ?</li>
<li>Quel est le nombre total des ventes et la valeur totale des ventes de tous magasins confondus ?</i>
</ol>

# QUestion 1
def mapper2(file_path):
  fichier = open(file_path, 'r')
  file_line = fichier.readlines()
  fichier.close()
  result = []

  for line in file_line:
    data = line.strip().split("\t")
    if len(data) == 6:
      date, time, store, item, montant, payment = data
      result.append(payment)
  
  return result

def reducer2(file_path):
  map = mapper2(file_path)
  fic = {}
  for elm in map:
    if elm in fic:
      fic.update({elm : int(fic[elm]) + 1})
    else:
      fic.update({elm : 0})
  
  for key in fic:
    print(key, "\t", fic[key]) 

file_path = '/root/myinput/purchases.txt'  
reducer2(file_path)

**PARTIE MAPPER**

In [76]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(payment + "\t1")

In [77]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper1.py', 'w') as f:
  f.write(In[76]) 
f.close()

In [79]:
!chmod u+rwx /content/mapper1.py

In [80]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper1.py

Amex	1
Visa	1
Cash	1
Discover	1
MasterCard	1
MasterCard	1
Visa	1
Cash	1
Discover	1
Visa	1


**PARTIE REDUCER**

In [81]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += int (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [82]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer1.py', 'w') as f:
  f.write(In[81]) 
f.close()

In [83]:
!chmod u+rwx /content/reducer1.py

Test du traitement distribué sur quelques enregistrements

In [84]:
!head -50 ~/myinput/purchases.txt | python3 /content/mapper1.py | sort | python3 /content/reducer1.py

Amex 	 10
Cash 	 10
Discover 	 8
MasterCard 	 9
Visa 	 13


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper1.py  -file /content/reducer1.py  -mapper 'python mapper1.py'  -reducer 'python reducer1.py'

In [86]:
!ls ~/myoutput

part-00000  _SUCCESS


In [87]:
!tail -n 30 ~/myoutput/part-00000

Amex 	 826535
Cash 	 828770
Discover 	 827426
MasterCard 	 828524
Visa 	 827221


#QUESTION2 : Chiffre d'affaire réalisé selon les jours de la semaine

In [88]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(date, "\t", amount)

In [89]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper2.py', 'w') as f:
  f.write(In[88]) 
f.close()

In [90]:
!chmod u+rwx /content/mapper2.py

In [91]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper2.py

2012-01-01 	 214.05
2012-01-01 	 153.57
2012-01-01 	 66.08
2012-01-01 	 493.51
2012-01-01 	 235.63
2012-01-01 	 247.18
2012-01-01 	 379.6
2012-01-01 	 296.8
2012-01-01 	 25.38
2012-01-01 	 213.88


In [92]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += float (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [93]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer2.py', 'w') as f:
  f.write(In[92]) 
f.close()

In [94]:
!chmod u+rwx /content/reducer2.py

In [95]:
!head -50 ~/myinput/purchases.txt | python3 /content/mapper2.py | sort | python3 /content/reducer2.py

2012-01-01  	 11259.819999999998


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper2.py  -file /content/reducer2.py  -mapper 'python mapper2.py'  -reducer 'python reducer2.py'

In [97]:
!ls ~/myoutput

part-00000  _SUCCESS


In [99]:
!tail -n 1000 ~/myoutput/part-00000

2012-01-01  	 2838071.4400000027
2012-01-02  	 2808888.290000001
2012-01-03  	 2877953.3099999856
2012-01-04  	 2829499.0200000056
2012-01-05  	 2838836.420000009
2012-01-06  	 2868368.749999997
2012-01-07  	 2843133.1600000085
2012-01-08  	 2903037.410000003
2012-01-09  	 2820050.500000003
2012-01-10  	 2815668.7099999976
2012-01-11  	 2863222.7200000184
2012-01-12  	 2837737.1500000083
2012-01-13  	 2840672.6899999925
2012-01-14  	 2844114.94
2012-01-15  	 2820928.5700000017
2012-01-16  	 2842436.4700000077
2012-01-17  	 2787413.2900000084
2012-01-18  	 2794907.0100000016
2012-01-19  	 2847904.5700000045
2012-01-20  	 2843726.099999992
2012-01-21  	 2812201.390000009
2012-01-22  	 2823239.8400000064
2012-01-23  	 2820093.1199999866
2012-01-24  	 2841298.409999998
2012-01-25  	 2854085.34
2012-01-26  	 2828425.459999995
2012-01-27  	 2846832.8199999966
2012-01-28  	 2805671.249999998
2012-01-29  	 2821061.889999988
2012-01-30  	 2853027.3700000015
2012-01-31  	 2847175.7900000084
2012

#QUESTION3 : la liste des magasins

In [101]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(store + "\t1")

In [104]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper3.py', 'w') as f:
  f.write(In[101]) 
f.close()

In [105]:
!chmod u+rwx /content/mapper3.py

In [ ]:
!head -20 ~/myinput/purchases.txt | python3 /content/mapper3.py

In [111]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotal = 0
oldKey = None

for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += int (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))

In [112]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer3.py', 'w') as f:
  f.write(In[111]) 
f.close()

In [109]:
!chmod u+rwx /content/reducer3.py

In [114]:
!head -500 ~/myinput/purchases.txt | python3 /content/mapper3.py | sort | python3 /content/reducer3.py

Albuquerque 	 8
Anaheim 	 8
Anchorage 	 7
Arlington 	 4
Atlanta 	 3
Aurora 	 3
Austin 	 8
Bakersfield 	 4
Baltimore 	 7
Baton Rouge 	 4
Birmingham 	 4
Boise 	 3
Boston 	 9
Buffalo 	 3
Chandler 	 4
Charlotte 	 4
Chesapeake 	 5
Chicago 	 5
Chula Vista 	 3
Cincinnati 	 4
Cleveland 	 3
Colorado Springs 	 3
Columbus 	 5
Corpus Christi 	 6
Dallas 	 4
Denver 	 4
Detroit 	 2
Durham 	 5
El Paso 	 3
Fort Wayne 	 2
Fort Worth 	 6
Fremont 	 5
Fresno 	 5
Garland 	 4
Gilbert 	 4
Glendale 	 5
Greensboro 	 10
Henderson 	 1
Hialeah 	 3
Honolulu 	 6
Houston 	 7
Indianapolis 	 5
Irvine 	 5
Irving 	 6
Jacksonville 	 5
Jersey City 	 7
Kansas City 	 7
Laredo 	 2
Las Vegas 	 4
Lexington 	 6
Lincoln 	 7
Long Beach 	 3
Los Angeles 	 3
Louisville 	 3
Lubbock 	 9
Madison 	 3
Memphis 	 5
Mesa 	 9
Miami 	 3
Milwaukee 	 3
Minneapolis 	 6
Nashville 	 8
Newark 	 4
New Orleans 	 5
New York 	 7
Norfolk 	 4
North Las Vegas 	 3
Oakland 	 4
Oklahoma City 	 4
Omaha 	 7
Orlando 	 2
Philadelphia 	 5
Phoenix 	 4
Pittsburgh 	 

# Références
**Réalisez des calculs distribués sur des données massives** : 
https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308656-familiarisez-vous-avec-hadoop

**Hadoop : la nouvelle infrastructure de gestion de données** : https://juvenal-chokogoue.developpez.com/tutoriels/hadoop-fonctionnement/

**MapReduce : comment l’utiliser pour le Big Data ?** : https://datascientest.com/mapreduce

**Calcul distribué: Hadoop et MapReduce** : http://b3d.bdpedia.fr/calculdistr.html

**Language Processing and Python** : https://www.nltk.org/book/ch01.html